In [75]:
#Importing read-sm-files.py
import ReadSMFIles
from Resource import Resource
from Task import Task
from Schedule import Schedule
import random
from typing import List, Tuple

#reading j30.sm/j301_1.sm file
sm_file = ReadSMFIles.SMFileParser.parse_sm_file("j30.sm/j301_1.sm")

print(sm_file[2].successors)


0        [2, 3, 4]
1      [6, 11, 15]
2       [7, 8, 13]
3       [5, 9, 10]
4             [20]
5             [30]
6             [27]
7     [12, 19, 27]
8             [14]
9         [16, 25]
10        [20, 26]
11            [14]
12        [17, 18]
13            [17]
14            [25]
15        [21, 22]
16            [22]
17        [20, 22]
18        [24, 29]
19        [23, 25]
20            [28]
21            [23]
22            [24]
23            [30]
24            [30]
25            [31]
26            [28]
27            [31]
28            [32]
29            [32]
30            [32]
31              []
Name: successors, dtype: object


In [72]:
#Creating resources
r1 = int(sm_file[4].R1[0])
r2 = int(sm_file[4].R2[0])
r3 = int(sm_file[4].R3[0])
r4 = int(sm_file[4].R4[0])

R1 = Resource('R1', r1)
R2 = Resource('R2', r2)
R3 = Resource('R3', r3)
R4 = Resource('R4', r4)

resources = [R1, R2, R3, R4]


In [73]:
#Creating jobs
jobs_enumerate = sm_file[3].jobnr
jobs_duration = sm_file[3].duration
jobs_resources = sm_file[3].resources
jobs_successors = sm_file[2].successors

jobs = [None for _ in jobs_enumerate]


for i in jobs_enumerate:
    jobs[i - 1] = Task(str(i), jobs_duration[i - 1])

for i in range(len(resources)):
    for j in range(len(jobs)):
        jobs[j].add_renewable_resource(resources[i], jobs_resources[j][i])

for i in range(len(jobs)):
    successors = jobs_successors[i]
    for j in successors:
        jobs[i].add_sucessor(jobs[j - 1])
    
for job in jobs:
    print([sucessor.name for sucessor in job.predecessors])

    

    



[]
['1']
['1']
['1']
['4']
['2']
['3']
['3']
['4']
['4']
['2']
['8']
['3']
['9', '12']
['2']
['10']
['13', '14']
['13']
['8']
['5', '11', '18']
['16']
['16', '17', '18']
['20', '22']
['19', '23']
['10', '15', '20']
['11']
['7', '8']
['21', '27']
['19']
['6', '24', '25']
['26', '28']
['29', '30', '31']



Genetic Algorithm



In [90]:
#Genetic Algorithm

#Creating random population
def create_population(population_size, jobs):
    population = []
    for i in range(population_size):
        schedule = Schedule()
        copy = jobs.copy()
        random.shuffle(copy)
        schedule.add_tasks(copy)
        population.append(schedule)
    
    return population


population = create_population(10, jobs)

for i in population:
    print([task.name for task in i.tasks])

['Dummy Source', '20', '30', '29', '12', '19', '26', '6', '9', '8', '13', '24', '25', '28', '32', '16', '27', '1', '22', '23', '7', '31', '3', '17', '11', '4', '21', '5', '14', '10', '2', '18', '15', 'Dummy Sink']
['Dummy Source', '9', '21', '11', '24', '13', '26', '17', '32', '31', '2', '15', '19', '5', '7', '4', '14', '23', '30', '8', '25', '29', '10', '28', '18', '16', '12', '20', '3', '22', '6', '27', '1', 'Dummy Sink']
['Dummy Source', '6', '14', '7', '22', '15', '10', '30', '31', '27', '24', '8', '20', '29', '28', '17', '21', '26', '19', '3', '16', '12', '5', '1', '9', '4', '18', '23', '2', '11', '25', '32', '13', 'Dummy Sink']
['Dummy Source', '29', '6', '2', '31', '12', '15', '14', '11', '10', '1', '24', '20', '27', '21', '13', '19', '5', '32', '25', '8', '3', '17', '23', '4', '28', '18', '22', '9', '26', '16', '30', '7', 'Dummy Sink']
['Dummy Source', '20', '24', '5', '13', '10', '21', '31', '22', '2', '16', '8', '9', '6', '12', '27', '7', '26', '23', '29', '28', '18', '14', '

Crossover

In [238]:

class OnePointCrossoverOperator:
    def __init__(self, crossover_rate: float = 0.8):
        self.crossover_rate = crossover_rate

    def __call__(
            self, parent1: Schedule, parent2: Schedule
    ) -> Tuple[Schedule, Schedule]:
        if random.random() > self.crossover_rate:
            return parent1, parent2

        n = len(parent1.tasks)
        crossover_point = random.randint(1, n - 3)

        child1 = Schedule()
        child2 = Schedule()

        parent1_tasks = parent1.tasks.copy()[1:-1]
        parent2_tasks = parent2.tasks.copy()[1:-1]

        child1.add_tasks(parent1_tasks[:crossover_point])
        for task in parent2.tasks:
            if task not in child1.tasks and task.name != 'Dummy Source' and task.name != 'Dummy Sink':
                child1.add_task(task)
        
        child2.add_tasks(parent2_tasks[:crossover_point])
        for task in parent1.tasks:
            if task not in child2.tasks and task.name != 'Dummy Source' and task.name != 'Dummy Sink':
                child2.add_task(task)

        return child1, child2


crossover = OnePointCrossoverOperator()

child1, child2 = crossover(population[0], population[1])

print([task.name for task in child1.tasks])
print([task.name for task in child2.tasks])
print(child1.tasks.size)



['Dummy Source', '20', '30', '9', '21', '11', '24', '13', '26', '17', '32', '31', '2', '15', '19', '5', '7', '4', '14', '23', '8', '25', '29', '10', '28', '18', '16', '12', '3', '22', '6', '27', '1', 'Dummy Sink']
['Dummy Source', '9', '21', '20', '30', '29', '12', '19', '26', '6', '8', '13', '24', '25', '28', '32', '16', '27', '1', '22', '23', '7', '31', '3', '17', '11', '4', '5', '14', '10', '2', '18', '15', 'Dummy Sink']
34


Parent Selection

In [88]:
class TournamentSelector:
    def __init__(self, tournament_size: int = 2):
        self.tournament_size = tournament_size

    def __call__(self, population: List[List[Task]]) -> List[Task]:
        selected = []
        for _ in range(len(population)):
            tournament = random.sample(population, self.tournament_size)
            winner = min(tournament, key=lambda x: x.makespan())
            selected.append(winner)
        return selected


selector = TournamentSelector(tournament_size=3)

selected = selector(population)

print([job.name for job in population[0]])

for i in range(len(selected)):
    print([job.name for job in selected[i]])
    print("\n")

AttributeError: 'list' object has no attribute 'makespan'